In [ ]:
%load_ext autoreload
%autoreload 2

# Create ColBERTRetriever

In [ ]:
from reprover.retrieval.colbert.model import ColBERTPremiseRetriever
from colbert.modeling.checkpoint import Checkpoint
from colbert.infra.config import ColBERTConfig

corpus_path = '../data/leandojo_benchmark_4/corpus.jsonl'
collection_path = '../data/leandojo_benchmark_4/random/colbert_collection.tsv'
checkpoint_path = '../checkpoints/colbertv2.0/'
num_negatives = 1
num_retrieved = 2

config = ColBERTConfig.load_from_checkpoint(checkpoint_path)
checkpoint = Checkpoint(checkpoint_path, colbert_config=config)

model = ColBERTPremiseRetriever(
    index_name='colbert_v1',
    experiment_name='lean',
    checkpoint_path_or_name=checkpoint_path,
    collection=collection_path,
    # config: Optional[Union[str, ColBERTConfig]] = None,
    index_root='../experiments/',
    num_retrieved=num_retrieved
)
model.load_corpus(corpus_path)

## Test corpus reindexing

In [ ]:
# model.reindex_corpus(64)

## Test retrieve() function

In [ ]:
from reprover.retrieval.datamodule import ColBERTRetrievalDataModule

data_path = "../data/leandojo_benchmark_4/random/"

data_module = ColBERTRetrievalDataModule(
    data_path,
    corpus_path,
    num_negatives=num_negatives,
    num_in_file_negatives=1,
    colbert_config=model.config,
    batch_size=1,
    eval_batch_size=1,
    max_seq_len=128,
    verbose=3,
    num_workers=0
)
data_module.setup()

example = data_module.ds_val.data[0]
ctx = example['context']


inputs = dict(
    state=[ctx.state],
    file_name=[ctx.path],
    theorem_full_name=[ctx.theorem_full_name],
    theorem_pos=[ctx.theorem_pos]
)

In [ ]:
ranking, premises, scores = model.retrieve(
    **inputs,
    k=2,
    reindex_batch_size=32,
    do_reindex=False
)

In [ ]:
premises, scores

## Test retrieve_from_preprocessed() function

In [ ]:
batch = next(iter(data_module.val_dataloader()))

In [ ]:
model.retrieve_from_preprocessed(batch)